# 1절. SQLite 데이터베이스 연결

## 1.1 SQLite와 파이썬
- SQLite 데이터 베이스는 별도의 서버프로세스 없이 SQL을 사용하여 DB액세스할 수 있도록 만든 간단한 디스크 기반 데이터베이스를 제공하는 C라이브러리
- SQLite는 프로토타입(시제품 나오기 전까지 제품)을 만들 때까지 사용. 정식 제품은 오라클이나 다른 DMBS를 이식.
- DB browser for SQLite(https://sqlitebrowser.org/dl)

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

## 1.2 데이터 베이스 연결

In [2]:
# DB 연결 객체 생성시, 파일이 없으면 파일 생성. 파일이 있으면 그 파일 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [3]:
# 커서 객체 반환. 커서는 SQL문을 실행시키고 결과데이터를 조회하는데 사용
cursor = conn.cursor()

In [4]:
cursor.execute("""
        CREATE TABLE MEMBER (
            NAME TEXT,
            AGE INT,
            EMAIL TEXT
        )
""")

In [5]:
cursor.execute("DROP TABLE MEMBER")

In [6]:
cursor.execute("""
        CREATE TABLE MEMBER (
            NAME TEXT,
            AGE INT,
            EMAIL TEXT
        )
""")

In [7]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'h@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('유길동',23,'y@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('마길동',30,'m@hong.com')")

In [8]:
conn.commit() # (반) conn.rollback()

In [9]:
cursor.execute('SELECT * FROM MEMBER') # 실행 결과는 cursor가 가르킴

In [10]:
# select문의 결과를 받는 함수 : 
## fetchall() - 결과를 모두 받을 때, 
## fetchone() - 결과를 한 행씩 받을 때, 
## fetchmay(n) - 결과를 n행 받을 때
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com'), ('유길동', 23, 'y@hong.com'), ('마길동', 30, 'm@hong.com')]


In [11]:
print(cursor.fetchall())

[]


In [12]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()
for member in members:
    print(member)

('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')
('마길동', 30, 'm@hong.com')


In [13]:
members[0]

('홍길동', 20, 'h@hong.com')

In [14]:
cursor.execute("SELECT * FROM MEMBER")
while True:
    member = cursor.fetchone()
    if member is None:
        break;
    print(member)

('홍길동', 20, 'h@hong.com')
('유길동', 23, 'y@hong.com')
('마길동', 30, 'm@hong.com')


In [15]:
cursor.execute("SELECT * FROM MEMBER")
for member in cursor.fetchmany(2):
    print(member[0], member[1], member[2])

홍길동 20 h@hong.com
유길동 23 y@hong.com


In [16]:
cursor.fetchmany(2)

[('마길동', 30, 'm@hong.com')]

In [17]:
cursor.close() # 생략 가능
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기
 - qmark, named(추천) 방법

In [18]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE name IN ('홍길동','김철수')")
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com')]


In [19]:
# 파라미터 사용하기 : qmark 방법 이용
cursor.execute("SELECT * FROM MEMBER WHERE name IN (?,?)",('홍길동','김철수'))
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com')]


In [20]:
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = (name1, name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
print(cursor.fetchall())

검색할 이름1 :홍기롱
검색할 이름2 :유길동
[('유길동', 23, 'y@hong.com')]


In [21]:
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = [name1, name2]
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
print(cursor.fetchall())

검색할 이름1 :홍길동
검색할 이름2 :마길동
[('홍길동', 20, 'h@hong.com'), ('마길동', 30, 'm@hong.com')]


In [24]:
# 파라미터 사용하기 : named 방법
name = input('검색하고자 하는 이름 : ')
cursor.execute("SELECT * FROM MEMBER where NAME=:name", {'name':name})
members = cursor.fetchall()
if len(members)==0 :
    print('해당 이름의 데이터가 없습니다')
else:
    print(cursor.fetchall())

검색하고자 하는 이름 : 아무개
해당 이름의 데이터가 없습니다


In [25]:
try:
    name = input('입력할 이름 ?')
    age = int(input('입력할 나이 ?'))    
    email = input('입력할 메일 ?')
except:
    print('나이를 잘못 입력하셨요. 초기값으로 20살')
    age = 20
    email = input('입력할 메일 ?')
cursor.execute("INSERT INTO MEMBER VALUES (:name,:age,:email)",
              {'name':name, 'age':age, 'email':email})

입력할 이름 ?신길동
입력할 나이 ?29
입력할 메일 ?shin@hong.com


In [26]:
cursor.execute('select * from member')
print(cursor.fetchall())

[('홍길동', 20, 'h@hong.com'), ('유길동', 23, 'y@hong.com'), ('마길동', 30, 'm@hong.com'), ('신길동', 29, 'shin@hong.com')]


In [27]:
cursor.close()
conn.close()

 # 2절. 오라클 데이터 베이스 연결

In [29]:
# 데이터 베이스 설정(DBMS가 있는 서버 주소, 포트번호, sid등을 설정)
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [30]:
conn = cx_Oracle.connect("scott","tiger", dsn=oracle_dsn)
conn
# 여기서 에러날 시 VC_redist.x64.exe 인스톨

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [31]:
# conn 얻어오는 방법(1)
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
conn = cx_Oracle.connect("scott","tiger", dsn=oracle_dsn)

In [32]:
conn.close()

In [33]:
# conn 얻어오는 방법(2)
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [34]:
# cursor 얻어오기
cursor = conn.cursor()

In [35]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [36]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [37]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
for e in cursor:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [38]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno = input('검색하고자 하는 부서 번호는 ?')
cursor.execute(sql, {'deptno':deptno})

for emp in cursor:
    print(emp)

검색하고자 하는 부서 번호는 ?30
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)


In [40]:
sql = "SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno = input('검색하고자 하는 부서 번호는 ?')
cursor.execute(sql, {'deptno':deptno})

emp = cursor.fetchall()
if len(emp)==0 :
    print('입력하신 부서번호의 데이터가 없습니다.')
else:
    for e in emp:
        print(e)

검색하고자 하는 부서 번호는 ?30
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)


In [41]:
print(emp)

[(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7900, 'JAMES', 'CLERK', 7698, datetime.datetime(1981, 12, 3, 0, 0), 950.0, None, 30)]


In [42]:
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플 리스트
data_df = pd.DataFrame(data) # 데이터 프레임
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [43]:
# 각 필드 특징 정보 (필드명, type, display_size, 내부크기, 정확도, scale, nullable)
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [44]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [45]:
data_df.columns = [row[0] for row in cursor.description]

In [46]:
data_df.columns

Index(['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO'], dtype='object')

In [47]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [48]:
data_df.tail()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30
10,7876,ADAMS,CLERK,7788.0,1983-01-12,1100.0,NaN,20
11,7900,JAMES,CLERK,7698.0,1981-12-03,950.0,NaN,30
12,7902,FORD,ANALYST,7566.0,1981-12-03,3000.0,NaN,20
13,7934,MILLER,CLERK,7782.0,1982-01-23,1300.0,NaN,10


In [49]:
cursor.close()
conn.close()

# 3절. MariaDB 데이터베이스 연결

In [51]:
import pymysql

In [ ]:
conn = pymysql.connect(host="localhost",
                      port=3306,
                      db="kimdb", user="root", passwd="mysql",
                      charset="utf8", autocommit=True)
conn

In [ ]:
cursor = conn.cursor()
sql = "select * from personal"
cursor.execute(sql)
result = cursor.fetchall()
for r in result:
    print(r)

In [ ]:
personal = pd.DataFrame(result)
personal.columns = [row[0] for row in cursor.description]
personal.head()

In [ ]:
cursor.close()
conn.close()